In [1]:
from postprocess_base import read_logs
import altair as alt
from glob import glob

runs = read_logs(glob('jeremy/*lassen*4.txt'))
runs['FE_nodes_per_compute_node'] = runs['num_unknowns'] / (runs['num_procs'] / runs['num_procs_node']) / runs['dof_per_node']
runs.head()

,file,backend,backend_memtype,hostname,test,num_procs,num_procs_node,degree,quadrature_pts,code,bp,case,num_unknowns,num_elem,dof_per_node,ksp_its,time_per_it,cg_iteration_dps,FE_nodes_per_compute_node
0,jeremy/lassen-1node-4.txt,/gpu/cuda/gen,device,lassen723,PETSc CEED Benchmark Problem 1,1,1,1,3,libCEED,1,scalar,5616,4692,1,5,0.000364,15439500.0,5616.0
1,jeremy/lassen-1node-4.txt,/gpu/cuda/gen,device,lassen723,PETSc CEED Benchmark Problem 2,1,1,1,3,libCEED,2,vector,16848,4692,3,5,0.000368,45782800.0,5616.0
2,jeremy/lassen-1node-4.txt,/gpu/cuda/gen,device,lassen723,PETSc CEED Benchmark Problem 3,1,1,1,3,libCEED,3,scalar,3872,4692,1,1,0.000508,7624400.0,3872.0
3,jeremy/lassen-1node-4.txt,/gpu/cuda/gen,device,lassen723,PETSc CEED Benchmark Problem 4,1,1,1,3,libCEED,4,vector,11616,4692,3,1,0.000521,22304700.0,3872.0
4,jeremy/lassen-1node-4.txt,/gpu/cuda/gen,device,lassen723,PETSc CEED Benchmark Problem 1,1,1,1,3,libCEED,1,scalar,10800,9384,1,5,0.000365,29591400.0,10800.0


In [9]:
highlight = alt.selection_single(
    on='mouseover',
    fields=['degree', 'time_per_it', 'backend'],
    nearest=True,
    empty='none',
)

bps_select = alt.selection_single(
    fields=['bp'],
)

base = alt.Chart(runs).encode(
    alt.Y('mdofs:Q', title='MDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns', 'file'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e6',
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5)),
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(2), alt.value(1))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', scale=alt.Scale(type='log'), title='Time per Iteration'),
    ) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True), title='FE Nodes per Compute Node'),
    )
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='CEED BP4 - Lassen (1x Volta V100), nvrtc-11.2')

alt.HConcatChart(...)